## General

In [1]:
import tensorflow as tf
import numpy as np
import random

epsilon = 0.00001
LR = 0.0002
alpha = 0.99

x_data = np.load('x_data.npy')
y_onehot = np.load('y_onehot.npy')
y_data = np.load('y_data.npy')

x_train = x_data[:400000]
y_train = y_onehot[:400000]

keep_prob = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32, [None, 100])
ys = tf.placeholder(tf.float32, [None, 5])

def add_layer(inputs, in_size, out_size, activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size], mean=0., stddev=1.))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, )
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
    fc_mean, fc_var = tf.nn.moments(Wx_plus_b,axes=[0],)
    scale = tf.Variable(tf.ones([out_size]))
    shift = tf.Variable(tf.zeros([out_size]))
    Wx_plus_b = tf.nn.batch_normalization(Wx_plus_b, fc_mean, fc_var, shift, scale, epsilon)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

l1 = add_layer(xs, 100, 200, activation_function=tf.nn.relu)
l2 = add_layer(l1, 200, 200, activation_function=tf.nn.relu)
output = add_layer(l2, 200, 5, activation_function=tf.nn.relu)
output = tf.nn.softmax(output)

loss = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(output), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(LR).minimize(loss)
init = tf.global_variables_initializer()

print "simple model loaded"

p = np.random.permutation(x_data.shape[0])
x_data, y_data, y_onehot = x_data[p], y_data[p], y_onehot[p]

print x_data.shape
print y_onehot.shape
print y_data.shape

x_train = x_data[:1500000]
y_train = y_onehot[:1500000]

sess = tf.Session()
sess.run(init)
for i in xrange(5):
    for j in xrange(15000):
        sess.run(train_step, feed_dict={xs: x_train[j*100:(j+1)*100], ys: y_train[j*100:(j+1)*100], keep_prob: 0.5})
        if (j+1)%1500==0:
            output_ = sess.run(output, feed_dict={xs: x_train, ys: y_train, keep_prob: 1})
            loss_value = sess.run(loss, feed_dict={xs: x_train, ys: y_train, keep_prob: 1})
            pred = np.argmax(output_, axis = 1)
            a = 0
            for ind in xrange(len(pred)):
                if pred[ind] == y_data[ind]-1:
                    a+=1
            correct_ratio = float(a)/len(pred)
            print "Round "+str(i)+": Batch "+str(j+1)+" loss: "+str(loss_value)+", correct_ratio "+" -- "+str(correct_ratio)
            LR*=alpha

simple model loaded
(1546730, 100)
(1546730, 5)
(1546730,)
Round 0: Batch 1500 loss: 1.63432, correct_ratio  -- 0.243325333333
Round 0: Batch 3000 loss: 1.60204, correct_ratio  -- 0.259771333333
Round 0: Batch 4500 loss: 1.58403, correct_ratio  -- 0.275820666667
Round 0: Batch 6000 loss: 1.57156, correct_ratio  -- 0.290654666667
Round 0: Batch 7500 loss: 1.56103, correct_ratio  -- 0.303117333333
Round 0: Batch 9000 loss: 1.55096, correct_ratio  -- 0.314953333333
Round 0: Batch 10500 loss: 1.54014, correct_ratio  -- 0.324615333333
Round 0: Batch 12000 loss: 1.52939, correct_ratio  -- 0.333784666667
Round 0: Batch 13500 loss: 1.51805, correct_ratio  -- 0.342012
Round 0: Batch 15000 loss: 1.50614, correct_ratio  -- 0.349026666667
Round 1: Batch 1500 loss: 1.49525, correct_ratio  -- 0.356026666667
Round 1: Batch 3000 loss: 1.48483, correct_ratio  -- 0.361221333333
Round 1: Batch 4500 loss: 1.47418, correct_ratio  -- 0.366866
Round 1: Batch 6000 loss: 1.46431, correct_ratio  -- 0.3723193333

In [3]:
x_test = x_data[1500000:]
y_test = y_data[1500000:]
yh_test = y_onehot[1500000:]
print x_test.shape
print y_test.shape
print yh_test.shape
output_ = sess.run(output, feed_dict={xs: x_test, ys: yh_test, keep_prob: 1})
pred = np.argmax(output_, axis = 1)
a = 0
for ind in xrange(len(pred)):
    if pred[ind] == y_test[ind]-1:
        a+=1
correct_ratio = float(a)/len(pred)
print "correct_ratio "+" -- "+str(correct_ratio)

(46730, 100)
(46730,)
(46730, 5)
correct_ratio  -- 0.405328482773


In [5]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
print "training..."
neigh.fit(x_train[:5000], y_data[:5000])
pred = neigh.predict(x_test)
print pred.shape
a = 0
for ind in xrange(len(pred)):
    if pred[ind] == y_test[ind]:
        a+=1
correct_ratio = float(a)/len(pred)
print "correct_ratio "+" -- "+str(correct_ratio)

training...
(46730,)
correct_ratio  -- 0.255189385834


In [6]:
neigh = KNeighborsClassifier(n_neighbors=5)
print "training..."
neigh.fit(x_train[:5000], y_data[:5000])
pred = neigh.predict(x_test)
print pred.shape
a = 0
for ind in xrange(len(pred)):
    if pred[ind] == y_test[ind]:
        a+=1
correct_ratio = float(a)/len(pred)
print "correct_ratio "+" -- "+str(correct_ratio)

training...
(46730,)
correct_ratio  -- 0.270511448748


In [7]:
neigh = KNeighborsClassifier(n_neighbors=10)
print "training..."
neigh.fit(x_train[:5000], y_data[:5000])
pred = neigh.predict(x_test)
print pred.shape
a = 0
for ind in xrange(len(pred)):
    if pred[ind] == y_test[ind]:
        a+=1
correct_ratio = float(a)/len(pred)
print "correct_ratio "+" -- "+str(correct_ratio)

training...
(46730,)
correct_ratio  -- 0.279199657608


In [9]:
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.fit(x_train[:10000], y_data[:10000])
pred = lin_clf.decision_function(x_test)
pred = np.argmax(pred, axis = 1)
print pred.shape
a = 0
for ind in xrange(len(pred)):
    if pred[ind] == y_test[ind]-1:
        a+=1
correct_ratio = float(a)/len(pred)
print "correct_ratio "+" -- "+str(correct_ratio)

(46730,)
correct_ratio  -- 0.399379413653


## Using LSTM

In [1]:
from tensorflow.contrib import rnn
import tensorflow as tf
import numpy as np
import random

x_data = np.load('x_data_lstm.npy')
y_onehot = np.load('y_onehot_lstm.npy')
y_data = np.load('y_data_lstm.npy')

x_data = np.transpose(x_data,(1,0,2))

x_train = x_data[:,:90000,:]
y_train = y_onehot[:90000]
x_test = x_data[:,90000:,:]
y_test = y_onehot[90000:]

LR = 0.01
alpha = 0.9995
epsilon = 0.000001
hidden = 128
n_epochs = 50

keep_prob = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32, [50, None, 100])
ys = tf.placeholder(tf.float32, [None, 5])

def add_layer(inputs, in_size, out_size, activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size], mean=0., stddev=1.))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, )
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
    fc_mean, fc_var = tf.nn.moments(Wx_plus_b,axes=[0],)
    scale = tf.Variable(tf.ones([out_size]))
    shift = tf.Variable(tf.zeros([out_size]))
    Wx_plus_b = tf.nn.batch_normalization(Wx_plus_b, fc_mean, fc_var, shift, scale, epsilon)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs
fw_cells = [rnn.BasicLSTMCell(hidden,state_is_tuple=True)]
bw_cells = [rnn.BasicLSTMCell(hidden,state_is_tuple=True)]
# xs_unstack = tf.unstack(xs,axis=0)
output_bi, output_state_fw, output_state_bw = rnn.stack_bidirectional_dynamic_rnn(fw_cells,bw_cells,xs,dtype=tf.float32,time_major=True)
features = tf.concat([output_state_fw[0][0], output_state_fw[0][1], output_state_bw[0][0], output_state_bw[0][1]], 1)

# l1 = add_layer(features, 400, 500, activation_function=tf.nn.relu)
# l2 = add_layer(l1, 500, 500, activation_function=tf.nn.relu)
l3 = add_layer(features, hidden*4, 5, activation_function=None)
output = tf.nn.softmax(l3)

loss = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(output), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(LR).minimize(loss)
init = tf.global_variables_initializer()


sess = tf.Session()
sess.run(init)
output_ = sess.run(output, feed_dict={xs: x_train, ys: y_train, keep_prob: 1})
pred = np.argmax(output_, axis = 1)
a = 0
for ind in xrange(len(pred)):
    if pred[ind] == y_data[ind]-1:
        a+=1
correct_ratio = float(a)/len(pred)
print "Initial, correct_ratio "+" -- "+str(correct_ratio)

saver = tf.train.Saver()
# saver.restore(sess, "lstm_mod/model.ckpt")

losses = []
accuracies = []

for i in xrange(n_epochs):
    for j in xrange(900):
        sess.run(train_step, feed_dict={xs: x_train[:, j*100:(j+1)*100, :], ys: y_train[j*100:(j+1)*100, :], keep_prob: 0.5})
        LR*=alpha
        if (j+1)%300 == 0:
#             print str(j+1)+"/900"
            output_ = sess.run(output, feed_dict={xs: x_train, ys: y_train, keep_prob: 1})
            loss_ = sess.run(loss, feed_dict={xs: x_train, ys: y_train, keep_prob: 1})
            pred = np.argmax(output_, axis = 1)
            a = 0
            for ind in xrange(len(pred)):
                if pred[ind] == y_data[ind]-1:
                    a+=1
            correct_ratio = float(a)/len(pred)
            print "Round "+str(i)+", Batch "+str(j+1)+", correct_ratio "+" -- "+str(correct_ratio)
            save_path = saver.save(sess, "lstm_mod/model.ckpt")
            print "Current Learning Rate = "+str(LR)
            print "Loss = "+str(loss_)
            losses.append(loss_)
            accuracies.append(correct_ratio)

Initial, correct_ratio  -- 0.212822222222
Round 0, Batch 300, correct_ratio  -- 0.268277777778
Current Learning Rate = 0.00860675689719
Loss = 1.59354
Round 0, Batch 600, correct_ratio  -- 0.323122222222
Current Learning Rate = 0.00740762642873
Loss = 1.55039
Round 0, Batch 900, correct_ratio  -- 0.337388888889
Current Learning Rate = 0.00637556398572
Loss = 1.53065
Round 1, Batch 300, correct_ratio  -- 0.354933333333
Current Learning Rate = 0.00548729293076
Loss = 1.52247
Round 1, Batch 600, correct_ratio  -- 0.368311111111
Current Learning Rate = 0.00472277962787
Loss = 1.4945
Round 1, Batch 900, correct_ratio  -- 0.365488888889
Current Learning Rate = 0.0040647816136
Loss = 1.49519
Round 2, Batch 300, correct_ratio  -- 0.3715
Current Learning Rate = 0.00349845871884
Loss = 1.49922
Round 2, Batch 600, correct_ratio  -- 0.382133333333
Current Learning Rate = 0.00301103837079
Loss = 1.48037
Round 2, Batch 900, correct_ratio  -- 0.387566666667
Current Learning Rate = 0.00259152752655
Lo

Current Learning Rate = 1.29707538615e-07
Loss = 1.39534
Round 25, Batch 300, correct_ratio  -- 0.459944444444
Current Learning Rate = 1.11636125259e-07
Loss = 1.37122
Round 25, Batch 600, correct_ratio  -- 0.458555555556
Current Learning Rate = 9.60824991047e-08
Loss = 1.36502
Round 25, Batch 900, correct_ratio  -- 0.458222222222
Current Learning Rate = 8.26958711868e-08
Loss = 1.37668
Round 26, Batch 300, correct_ratio  -- 0.4602
Current Learning Rate = 7.11743259706e-08
Loss = 1.36779
Round 26, Batch 600, correct_ratio  -- 0.461677777778
Current Learning Rate = 6.1258012095e-08
Loss = 1.36214
Round 26, Batch 900, correct_ratio  -- 0.461888888889
Current Learning Rate = 5.27232818107e-08
Loss = 1.37419
Round 27, Batch 300, correct_ratio  -- 0.463677777778
Current Learning Rate = 4.53776469366e-08
Loss = 1.36323
Round 27, Batch 600, correct_ratio  -- 0.456655555556
Current Learning Rate = 3.9055437575e-08
Loss = 1.36593
Round 27, Batch 900, correct_ratio  -- 0.453033333333
Current Lea

Current Learning Rate = 1.68240455735e-12
Loss = 1.36651


In [6]:
x_test = x_data[:,90000:,:]
y_test = y_onehot[90000:]

output_ = sess.run(output, feed_dict={xs: x_test, ys: y_test, keep_prob: 1})
print output.shape
pred = np.argmax(output_, axis = 1)
print pred.shape
a = 0
for ind in xrange(len(pred)):
    if pred[ind] == y_data[90000+ind]-1:
        a+=1
correct_ratio = float(a)/len(pred)
print "correct_ratio "+" -- "+str(correct_ratio)

(?, 5)
(10390,)
correct_ratio  -- 0.466566666667


In [8]:
print x_test.shape
print y_test.shape

(50, 10390, 100)
(10390, 5)


In [15]:
import numpy as np
import random

word_vectors = {}

glove_size = 0
with open('data/glove.6B.100d.txt', 'r') as f:
    for line in f:
        glove_size+=1
print glove_size

review_size = 0
with open('data/reviews_Books_5.json', 'r') as f:
    for line in f:
        review_size+=1
print review_size

line_index = 0
with open('data/glove.6B.100d.txt', 'r') as f:
    for line in f:
        line_index+=1
        if line_index%100000==0:
            print "Reading word_vectors... "+str(line_index)+"/"+str(glove_size)
        line_list = line.split()
        word = line_list[0]
        vec = [float(v) for v in line_list[1:]]
        assert len(vec) == 100
        word_vectors[word] = vec

print len(word_vectors)

x_data = []
y_data = []

padding_vec = np.zeros(100)

n5 = 0
n4 = 0
n3 = 0
n2 = 0
n1 = 0
texts = []

with open('data/reviews_Books_5.json', 'r') as f:
    for line in f:
        if n1+n2+n3+n4+n5==200:
            break
        if "reviewText" not in line or "overall" not in line:
            continue
        review_start = line.index("\"reviewText\":")+15
        review_end = line.index("\"overall\":")-3
        if review_end-review_start<50:
            continue
        review = line[review_start:review_end]
        texts.append(review)
        overall = int(line[review_end+14:review_end+15])
        if overall == 5 and n5 == 40:
            continue
        if overall == 4 and n4 == 40:
            continue
        if overall == 3 and n3 == 40:
            continue
        if overall == 2 and n2 == 40:
            continue
        if overall == 1 and n1 == 40:
            continue
        if overall == 5:
            n5+=1
        if overall == 4:
            n4+=1
        if overall == 3:
            n3+=1
        if overall == 2:
            n2+=1
        if overall == 1:
            n1+=1 

        for i in xrange(len(review)):
            if not review[i].isalpha():
                review = review[:i]+" "+review[i:]
        review = review.split()
        
        word_sequence = []
        for i in xrange(50):
            if i>=len(review):
                word_vec = padding_vec
            else:
                word = review[i].lower()
                if word in word_vectors:
                    word_vec = np.array(word_vectors[word])
                else:
                    word_vec = np.random.rand(100)
            word_sequence.append(word_vec)
        x_data.append(word_sequence)
        y_data.append(overall)
x_data = np.array(x_data)
x_data = np.transpose(x_data,(1,0,2))
y_data = np.array(y_data)
y_onehot = np.zeros((len(y_data), 5))
y_onehot[np.arange(len(y_data)), y_data-1] = 1

print x_data.shape
print y_data.shape
print y_onehot.shape

output_ = sess.run(output, feed_dict={xs: x_data, ys: y_onehot, keep_prob: 1})

print "------------------"
for i in xrange(200):
    print texts[i]
    print output_[i]
    print y_data[i]
    print "------------------"

400000
8898041
Reading word_vectors... 100000/400000
Reading word_vectors... 200000/400000
Reading word_vectors... 300000/400000
Reading word_vectors... 400000/400000
400000
(50, 200, 100)
(200,)
(200, 5)
------------------
Spiritually and mentally inspiring! A book that allows you to question your morals and will help you discover who you really are!
[ 0.27335832  0.10917691  0.15325348  0.22109373  0.24311759]
5
------------------
This is one my must have books. It is a masterpiece of spirituality. I'll be the first to admit, its literary quality isn't much. It is rather simplistically written, but the message behind it is so powerful that you have to read it. It will take you to enlightenment.
[ 0.12155917  0.0959346   0.17722087  0.1825022   0.4227832 ]
5
------------------
This book provides a reflection that you can apply to your own life.And, a way for you to try and assess whether you are truly doing the right thing and making the most of your short time on this plane.
[ 0.1699

For one who would prefer Truth coming from a source other than the King James Version, \"The Prophet\" is written with the voice of wisdom and the beauty of poetry. If there is a Black and White 'Truth,' one could easily seek it here in almost all the verses.
[ 0.20788543  0.22864704  0.20763351  0.21275753  0.14307646]
3
------------------
Gibran wrote the ultimate way of seeing life and living in it. the importance of each relationship. The book was written to be relatable to every reader, all the subjects are mondain. The availability of this book is a great advantage that I don't think people are aware of. It's an hour long to read, but a lifetime to understand, if you read it you'll remember it in every new life matter or relationship.Buy it and keep it on the shelve for reference, it's an answer to everything. and read it carefully.
[ 0.17155445  0.15790468  0.18032433  0.21240376  0.27781281]
3
------------------
Gibran was a great author and in this book he truly shines. I reco